In [1]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import time

%matplotlib inline

In [18]:
class Root():
    
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Delivery robot Neural Network")
        self.root.minsize(350, 350)
        self.root.maxsize(350, 350)
        #para mostrar la foto
        self.label_text1 = tk.Label(self.root,text="Input Image")
        self.label_text1.grid(column=0,row=1,padx= 100, pady= 20)
        self.label_frame_1 = tk.LabelFrame(self.root, text="")
        self.label_frame_1.grid(column=0, row=3, padx=100)
        
        self.label_frame_2 = tk.LabelFrame(self.root, text="")
        self.label_frame_3 = tk.LabelFrame(self.root, text="Prediction")
        self.label_frame_3.grid(column=0, row=6, padx=100)
        #self.root.iconphoto(False, tk.PhotoImage(file='../images/robot.png'))
        #cargar el modelo tflite
        self.interpreter = tf.lite.Interpreter("../model_test_tflite.tflite")
        self.interpreter.allocate_tensors()
        self.input_vars = self.interpreter.get_input_details()
        self.output_vars = self.interpreter.get_output_details()
        self.input_shape = self.input_vars[0]["shape"]
        self.ruta_guardar = "../data/tf_lite_images/"
        self.count_photos = 0
        
        self.pre_btn = tk.Button(self.root, text="Run robot", command=lambda: self.make_predictions())
        self.pre_btn.grid(column=0, row=4, pady=15)
        
        self.v_class = tk.StringVar()
        self.v_proba = tk.StringVar()
        self.count_initial_predict = 0
        
        #self.pre_btn.bind('<Button-1>', lambda event: self.make_predictions())
        
        self.root.mainloop()
        
    def take_a_photo(self):
        cam = cv2.VideoCapture(0)
        flag_photo, img = cam.read()
        if flag_photo:
            cv2.namedWindow("cam-test", cv2.WINDOW_AUTOSIZE)
            #waitKey(0)
            cv2.destroyWindow("cam-test") 
            cam.release() #dejar de capturar la cámara
            return img
            
    def make_predictions(self):
        #tomamos la foto
        img_show = self.take_a_photo()
        
        #por defecto opencv toma los colores como BGR, hay que convertir la imagen en RGB
        img_colors = cv2.cvtColor(img_show, cv2.COLOR_RGB2BGR)
        img_PIL = Image.fromarray(img_colors).resize((self.input_shape[1], self.input_shape[2]), Image.BICUBIC)
        render = ImageTk.PhotoImage(img_PIL)
        #hacemos la predicción
        input_image = np.expand_dims(np.array(img_PIL, dtype=np.float32), axis=0)
        self.interpreter.set_tensor(self.input_vars[0]["index"], input_image)
        self.interpreter.invoke()
        prediction = self.interpreter.get_tensor(self.output_vars[0]["index"])[0]
        class_pre = np.argmax(prediction)
        proba = prediction[class_pre]
        #se guarda la imágen con la clase que predijo
        self.photo_name = "image_"+str(self.count_photos)+"_class_"+str(class_pre)+".jpg"
        cv2.imwrite(self.ruta_guardar + self.photo_name, img_show)
        
        self.count_photos += 1
        
        if self.count_initial_predict == 0:
            self.img_1 = tk.Label(self.label_frame_1, image=render)
            self.img_1.configure(image=render)
            self.img_1.image=render
            self.img_1.grid(column=0,row=3)

            #pintar las predicciones
            self.v_class.set("Class: " + str(class_pre))
            self.v_proba.set("Probability: " + str(proba) + "%")
            #texto de predicción
            lb_class = tk.Label(self.label_frame_3, textvariable=self.v_class)
            lb_class.grid(column=0,row=5)
            lb_proba = tk.Label(self.label_frame_3, textvariable=self.v_proba)
            lb_proba.grid(column=0,row=6)

            self.count_initial_predict += 1
        else:
            self.img_1.configure(image=render)
            self.img_1.image=render
            self.v_class.set("Class: " + str(class_pre))
            self.v_proba.set("Probability: " + str(proba) + "%")
        
        self.root.after(5000, self.make_predictions)
            

In [68]:
%%writefile 'script_to_send_images.py'

import paramiko
from scp import SCPClient
import datetime
import time

def create_SSH_Client(host, port, user, password_name, password_path):
    client = paramiko.SSHClient()
    client.load_system_host_keys()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(host, port, user, password_name, key_filename=password_path)
    return client

while True:
    hora = datetime.datetime.now().hour
    if hora == 24:
        #se cargan las imaǵenes al servidor
        ssh = create_SSH_Client("18.116.37.95", 22, "ubuntu","Deliveryrobotpassword.pem",
                     "../../../proyectos_ai/delivery_robot/Deliveryrobotpassword.pem")
        scp = SCPClient(ssh.get_transport())
        scp.put('../data/tf_lite_images', recursive=True,
                remote_path='/home/ubuntu/delivery_robot_project/static/')
        #se cierra la conexión al terminar
        ssh.close()
    time.sleep(60*60) #se para la ejecución por una hora


Writing script_to_send_images.py
